In [ ]:
using QuantumOptics
using Einsum

In [91]:
basis = NLevelBasis(2) 

NLevel(N=2)

In [92]:
t = -1

H1 = t * ( transition(basis,1,2) + transition(basis,2,1) )

Operator(dim=2x2)
  basis: NLevel(N=2)sparse([2, 1], [1, 2], ComplexF64[-1.0 - 0.0im, -1.0 - 0.0im], 2, 2)

In [93]:
E1, U1 = eigenstates(dense(H1))

([-1.0, 1.0], Ket{NLevelBasis{Int64}, Vector{ComplexF64}}[Ket(dim=2)
  basis: NLevel(N=2)
 -0.7071067811865475 + 0.0im
 -0.7071067811865475 + 0.0im, Ket(dim=2)
  basis: NLevel(N=2)
 -0.7071067811865475 + 0.0im
  0.7071067811865475 + 0.0im])

In [95]:
# projection
substates = 1:2
basis_tilde = SubspaceBasis(basis, U1[substates])

Subspace(superbasis=NLevel(N=2), states:2)

In [96]:
projector(basis_tilde, basis)

Operator(dim=2x2)
  basis left:  Subspace(superbasis=NLevel(N=2), states:2)
  basis right: NLevel(N=2)
 -0.707107-0.0im  -0.707107-0.0im
 -0.707107-0.0im   0.707107-0.0im

In [97]:
projector(basis, basis_tilde)

Operator(dim=2x2)
  basis left:  NLevel(N=2)
  basis right: Subspace(superbasis=NLevel(N=2), states:2)
 -0.707107+0.0im  -0.707107+0.0im
 -0.707107+0.0im   0.707107+0.0im

In [98]:
P1 = projector(basis, basis_tilde)
P1_dag = dagger(P1)

Operator(dim=2x2)
  basis left:  Subspace(superbasis=NLevel(N=2), states:2)
  basis right: NLevel(N=2)
 -0.707107-0.0im  -0.707107-0.0im
 -0.707107-0.0im   0.707107-0.0im

In [99]:
H1_sub = P1_dag*H1*P1

Operator(dim=2x2)
  basis: Subspace(superbasis=NLevel(N=2), states:2)
 -1.0+0.0im  0.0+0.0im
  0.0+0.0im  1.0+0.0im

In [100]:
# many-body basis
PN = 2
states_mb = bosonstates(basis, PN)

3-element Vector{Vector{Int64}}:
 [2, 0]
 [1, 1]
 [0, 2]

In [101]:
basis_mb = ManyBodyBasis(basis, states_mb)

ManyBody(onebodybasis=NLevel(N=2), states:3)

In [102]:
H1_mb = manybodyoperator(basis_mb, H1)

Operator(dim=3x3)
  basis: ManyBody(onebodybasis=NLevel(N=2), states:3)sparse([2, 1, 3, 2], [1, 2, 2, 3], ComplexF64[-1.4142135623730951 + 0.0im, -1.4142135623730951 + 0.0im, -1.4142135623730951 + 0.0im, -1.4142135623730951 + 0.0im], 3, 3)

In [103]:
# Hubbard interaction
U0 = 2
n1 = number(basis_mb, 1)
n2 = number(basis_mb, 2)
Id = identityoperator(basis_mb)
Vint_mb = U0/2*(n1*(n1-Id) + n2*(n2-Id))

Operator(dim=3x3)
  basis: ManyBody(onebodybasis=NLevel(N=2), states:3)sparse([1, 3], [1, 3], ComplexF64[2.0 + 0.0im, 2.0 + 0.0im], 3, 3)

In [104]:
H_mb = H1_mb + Vint_mb

Operator(dim=3x3)
  basis: ManyBody(onebodybasis=NLevel(N=2), states:3)sparse([1, 2, 1, 3, 2, 3], [1, 1, 2, 2, 3, 3], ComplexF64[2.0 + 0.0im, -1.4142135623730951 + 0.0im, -1.4142135623730951 + 0.0im, -1.4142135623730951 + 0.0im, -1.4142135623730951 + 0.0im, 2.0 + 0.0im], 3, 3)

In [105]:
dense( H_mb )

Operator(dim=3x3)
  basis: ManyBody(onebodybasis=NLevel(N=2), states:3)
      2.0+0.0im  -1.41421+0.0im       0.0+0.0im
 -1.41421+0.0im       0.0+0.0im  -1.41421+0.0im
      0.0+0.0im  -1.41421+0.0im       2.0+0.0im

In [106]:
eigenenergies( dense(H_mb) )

3-element Vector{Float64}:
 -1.2360679774997894
  2.0
  3.23606797749979

In [107]:
# alternatively construct Vint from the 2-body matrix elements
# transition(basis,i,k)⊗transition(basis,j,l) goes with V_ijkl 

In [108]:
V2  = U0/2*transition(basis,1,1)⊗transition(basis,1,1) 
V2 += U0/2*transition(basis,2,2)⊗transition(basis,2,2) 

Operator(dim=4x4)
  basis: [NLevel(N=2) ⊗ NLevel(N=2)]sparse([1, 4], [1, 4], ComplexF64[1.0 + 0.0im, 1.0 + 0.0im], 4, 4)

In [109]:
V2.data

4×4 SparseArrays.SparseMatrixCSC{ComplexF64, Int64} with 2 stored entries:
 1.0+0.0im      ⋅          ⋅          ⋅    
     ⋅          ⋅          ⋅          ⋅    
     ⋅          ⋅          ⋅          ⋅    
     ⋅          ⋅          ⋅      1.0+0.0im

In [110]:
V2_mb = manybodyoperator(basis_mb, V2)

Operator(dim=3x3)
  basis: ManyBody(onebodybasis=NLevel(N=2), states:3)sparse([1, 3], [1, 3], ComplexF64[2.0000000000000004 + 0.0im, 2.0000000000000004 + 0.0im], 3, 3)

In [111]:
isapprox(V2_mb, Vint_mb)

true

In [112]:
# compare the following for indexing the 2-ptl states
transition(basis,1,2)⊗transition(basis,1,1), transition(basis,1,1)⊗transition(basis,1,2)

(Operator(dim=4x4)
  basis: [NLevel(N=2) ⊗ NLevel(N=2)]sparse([1], [2], ComplexF64[1.0 + 0.0im], 4, 4), Operator(dim=4x4)
  basis: [NLevel(N=2) ⊗ NLevel(N=2)]sparse([1], [3], ComplexF64[1.0 + 0.0im], 4, 4))

In [113]:
P1M = P1.data
P1_dagM = P1_dag.data

2×2 adjoint(::Matrix{ComplexF64}) with eltype ComplexF64:
 -0.707107-0.0im  -0.707107-0.0im
 -0.707107-0.0im   0.707107-0.0im

In [114]:
H1M = H1.data

2×2 SparseArrays.SparseMatrixCSC{ComplexF64, Int64} with 2 stored entries:
      ⋅      -1.0-0.0im
 -1.0-0.0im       ⋅    

In [115]:
V2M = V2.data

4×4 SparseArrays.SparseMatrixCSC{ComplexF64, Int64} with 2 stored entries:
 1.0+0.0im      ⋅          ⋅          ⋅    
     ⋅          ⋅          ⋅          ⋅    
     ⋅          ⋅          ⋅          ⋅    
     ⋅          ⋅          ⋅      1.0+0.0im

In [116]:
P2M = (P1⊗P1).data
P2_dagM = (P1_dag⊗P1_dag).data

4×4 Matrix{ComplexF64}:
 0.5+0.0im   0.5+0.0im   0.5+0.0im   0.5+0.0im
 0.5+0.0im  -0.5+0.0im   0.5+0.0im  -0.5+0.0im
 0.5+0.0im   0.5+0.0im  -0.5+0.0im  -0.5+0.0im
 0.5+0.0im  -0.5+0.0im  -0.5+0.0im   0.5-0.0im

In [117]:
@einsum H1_tildeM[i,j] := P1_dagM[i,k] * H1M[k,l] * P1M[l,j];
@einsum V2_tildeM[i2,j2] :=  P2_dagM[i2,k2] * V2M[k2,l2] * P2M[l2,j2];
@einsum U4[k,l,m,n] := P1M[k,i] * P1M[l,i] * P1_dagM[i,m] * P1_dagM[i,n];

In [118]:
H1_tilde = Operator(basis_tilde, H1_tildeM)

Operator(dim=2x2)
  basis: Subspace(superbasis=NLevel(N=2), states:2)
 -1.0+0.0im  0.0+0.0im
  0.0+0.0im  1.0+0.0im

In [119]:
basis_mb_tilde  = ManyBodyBasis(basis_tilde, states_mb)

ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=2), states:2), states:3)

In [120]:
H_NI_tilde = manybodyoperator(basis_mb_tilde, H1_tilde)

Operator(dim=3x3)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=2), states:2), states:3)
 -2.0+0.0im  0.0+0.0im  0.0+0.0im
  0.0+0.0im  0.0+0.0im  0.0+0.0im
  0.0+0.0im  0.0+0.0im  2.0+0.0im

In [121]:
basis_tilde2 = basis_tilde⊗basis_tilde

[Subspace(superbasis=NLevel(N=2), states:2) ⊗ Subspace(superbasis=NLevel(N=2), states:2)]

In [122]:
Vint_tilde = Operator(basis_tilde2, U0/2*reshape(U4,4,4))

Operator(dim=4x4)
  basis: [Subspace(superbasis=NLevel(N=2), states:2) ⊗ Subspace(superbasis=NLevel(N=2), states:2)]
 0.5+0.0im  0.0+0.0im  0.0+0.0im  0.5+0.0im
 0.0+0.0im  0.5+0.0im  0.5+0.0im  0.0+0.0im
 0.0+0.0im  0.5+0.0im  0.5+0.0im  0.0+0.0im
 0.5+0.0im  0.0+0.0im  0.0+0.0im  0.5+0.0im

In [123]:
V2_tilde = Operator(basis_tilde2, V2_tildeM)

Operator(dim=4x4)
  basis: [Subspace(superbasis=NLevel(N=2), states:2) ⊗ Subspace(superbasis=NLevel(N=2), states:2)]
 0.5+0.0im  0.0+0.0im  0.0+0.0im  0.5+0.0im
 0.0+0.0im  0.5+0.0im  0.5+0.0im  0.0+0.0im
 0.0+0.0im  0.5+0.0im  0.5+0.0im  0.0+0.0im
 0.5+0.0im  0.0+0.0im  0.0+0.0im  0.5+0.0im

In [128]:
isapprox(Vint_tilde, V2_tilde)

true

In [124]:
Vint_tilde_mb = manybodyoperator(basis_mb_tilde, Vint_tilde)

Operator(dim=3x3)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=2), states:2), states:3)
 1.0+0.0im  0.0+0.0im  1.0+0.0im
 0.0+0.0im  2.0+0.0im  0.0+0.0im
 1.0+0.0im  0.0+0.0im  1.0+0.0im

In [125]:
Vint_tilde_mb = manybodyoperator(basis_mb_tilde, V2_tilde)

Operator(dim=3x3)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=2), states:2), states:3)
 1.0+0.0im  0.0+0.0im  1.0+0.0im
 0.0+0.0im  2.0+0.0im  0.0+0.0im
 1.0+0.0im  0.0+0.0im  1.0+0.0im

In [126]:
H_mb_tilde = H_NI_tilde + Vint_tilde_mb

Operator(dim=3x3)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=2), states:2), states:3)
 -1.0+0.0im  0.0+0.0im  1.0+0.0im
  0.0+0.0im  2.0+0.0im  0.0+0.0im
  1.0+0.0im  0.0+0.0im  3.0+0.0im

In [127]:
eigenenergies( dense(H_mb_tilde) )

3-element Vector{Float64}:
 -1.2360679774997896
  1.999999999999999
  3.23606797749979